## Notes

We are likely overfitting here due to low sample size. For now, we keep this example for future reference as the procedure is valid.

### Imports

In [ ]:
import logging
import sys
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

# Add the parent directory to the path so we can import the modules
# note that in /notebooks, use sys.path.append(os.path.abspath(".."))
sys.path.append(os.path.abspath("../.."))


from scripts.database import get_session, load_data_to_db
from scripts.utils import load_config, setup_logging
from models import NetflixDataset


### Config and Logging Setup

In [ ]:
try:
    config = load_config()
    setup_logging(config['paths']['log_path'])
    logging.info("Starting the data analysis project.")
except Exception as e:
    logging.error(f"Failed to load config or setup logging: {e}")
    raise

### Database Session

In [ ]:
try:
    session = get_session()
    logging.info("Database session created successfully.")
except Exception as e:
    logging.error(f"Failed to create database session: {e}")
    raise

### Query netflix Data

In [ ]:
try:
    show_data = session.query(NetflixDataset).all()
    data = [show.__dict__ for show in show_data]
    for item in data:
        item.pop('_sa_instance_state', None)
    
    show_df = pd.DataFrame(data)
    print(show_df.head())
except Exception as e:
    logging.error(f"Failed to query show data: {e}")
    raise


### Feature Engineering and Mutual Information

In [ ]:
X = show_df['text']
y = show_df['type']

# Text vectorization
vectorizer = TfidfVectorizer(max_features=5000)
X_vectorized = vectorizer.fit_transform(X)

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X_vectorized, y, test_size=0.2, random_state=42)

# Train classifier
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train, y_train)

# Predict
y_pred = clf.predict(X_test)

# Evaluation
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))